# Setup

In [1]:
%load_ext autoreload
%autoreload 2
%aimport

%matplotlib inline 

Modules to reload:
all-except-skipped

Modules to skip:



In [2]:
import matplotlib.pyplot as plt
import torch

from bliss.inference import SDSSFrame
from bliss.datasets import sdss
from bliss.inference import reconstruct_scene_at_coordinates
from case_studies.sdss_galaxies.plots import load_models


from astropy.table import Table

import plotly.express as px
import plotly.graph_objects as go

Global seed set to 40


In [5]:
# check GPU is configured correctly
device = torch.device('cuda:0')
!echo $CUDA_VISIBLE_DEVICES

5


# Get Config

In [ ]:
# load models
from hydra import compose, initialize
from hydra.utils import instantiate
from bliss.encoder import Encoder

with initialize(config_path="../../config"):
    cfg = compose("config", overrides=[])
    

# Run

In [7]:
cfg.datasets.sdss_galaxies

{'_target_': 'bliss.datasets.galsim_galaxies.SingleGalsimGalaxies', 'prior': {'_target_': 'bliss.datasets.galsim_galaxies.GalsimGalaxyPrior', 'flux_sample': 'pareto', 'min_flux': 622.0, 'max_flux': 1000000.0, 'alpha': 0.47, 'a_sample': 'gamma', 'a_concentration': 0.39330758068481686, 'a_loc': 0.8371888967872619, 'a_scale': 4.432725319432478, 'a_bulge_disk_ratio': 2.0}, 'decoder': {'_target_': 'bliss.datasets.galsim_galaxies.GalsimGalaxyDecoder', 'n_bands': 1, 'slen': 53, 'pixel_scale': 0.396, 'psf_image_file': '${paths.sdss}/psField-000094-1-0012-PSF-image.npy'}, 'background': {'_target_': 'bliss.datasets.background.ConstantBackground', 'background': [865.0]}, 'num_workers': 0, 'batch_size': 64, 'n_batches': 10}

In [6]:
ds = instantiate(cfg.datasets.sdss_galaxies)

InstantiationException: Error in call to target 'bliss.datasets.galsim_galaxies.GalsimGalaxyDecoder':
FileNotFoundError(2, 'No such file or directory')
full_key: datasets.sdss_galaxies.decoder

In [ ]:
# Load data

# load sdss data
sdss_dir = '/home/imendoza/bliss/data/sdss/'
pixel_scale = 0.393
coadd_file = "/home/imendoza/bliss/data/coadd_catalog_94_1_12.fits"
frame = SDSSFrame(sdss_dir, pixel_scale, coadd_file)

In [4]:
# load models
from hydra import compose, initialize
from hydra.utils import instantiate
from bliss.encoder import Encoder

with initialize(config_path="../../config"):
    cfg = compose("config", overrides=[])
    
    
enc, dec = load_models(cfg, device)
bp = enc.border_padding
torch.cuda.empty_cache()

NameError: name 'device' is not defined